### catalogo de datos madrid

In [2]:
#parse dcat xml
import xml.etree.ElementTree as ET

ns = {
    'foaf': "http://xmlns.com/foaf/0.1/",
    'skos': "http://datos.gob.es/kos/sector-publico/sector/",
    'dcat': "http://www.w3.org/ns/dcat#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'dct': "http://purl.org/dc/terms/",
    'dc': "http://purl.org/dc/elements/1.1/",
    'time': "http://www.w3.org/2006/time#",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'j.0': "http://www.w3.org/1999/xhtml/vocab#",
    'j.1': "http://a9.com/-/spec/opensearch/1.1/",
    'api': "http://purl.org/linked-data/api/vocab#",
}





In [3]:
#desde fichero
with open('../data/catalog.rdf') as rdf:
    tree = ET.parse(rdf)
    root = tree.getroot()

In [3]:
#desde url
import urllib.request

with urllib.request.urlopen('https://datos.madrid.es/egob/catalogo.rdf') as f:
    tree = ET.parse(f)
    root = tree.getroot()

In [ ]:
for child in root:
    print(child.tag)

In [ ]:
catalog =root.find('dcat:Catalog', ns)
print(catalog.tag)

for dataset in catalog.findall('dcat:dataset', ns):
    print(dataset.tag)

In [ ]:
for dataset in root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset/dct:identifier', ns):
    print(dataset.text)

len(root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset/dct:identifier', ns))

In [ ]:
for dataset in root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset', ns):
    #print(dataset.find('dct:identifier',ns).text, dataset.find('dct:title',ns).text)
    for distribution in dataset.findall('dcat:distribution/dcat:Distribution', ns):
        print( dataset.find('dct:identifier',ns).text, 
              dataset.find('dct:title',ns).text,
              distribution.find('dct:title', ns).text if distribution.find('dct:title', ns) else '', 
              distribution.find('dcat:mediaType', ns).text, 
              distribution.find('dcat:accessURL', ns).text,
              sep='; ')

    

In [ ]:
from datetime import datetime
ds_list = [{'indentifier': dataset.find('dct:identifier', ns).text,
            'dataset_title': dataset.find('dct:title', ns).text
            , 'theme': dataset.find('dcat:theme', ns).attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
            , 'keywords': dataset.find('dcat:keyword', ns).text
            , 'issued':  datetime.strptime(dataset.find('dct:issued', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'modified': datetime.strptime(dataset.find('dct:modified', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'frequency': dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns).text if  dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns) is not None else None
            , 'distribution_title': distribution.find('dct:title', ns).text if distribution.find('dct:title', ns) is not None else None
            , 'mediaType': distribution.find('dcat:mediaType', ns).text
            , 'accessURL': distribution.find('dcat:accessURL', ns).text
            }
           for dataset
           in root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset', ns)
           for distribution
           in dataset.findall('dcat:distribution/dcat:Distribution', ns)

           ]

ds_list

In [ ]:
import polars as pl

dfp = pl.DataFrame(ds_list)
dfp.write_csv('../data/catalogo2.csv', separator=';', include_bom=True)

In [ ]:
import duckdb

duckdb.sql("""
           SELECT * 
           FROM dfp 
         -- where frequency = 'tiemporeal'
           order by modified desc """)

In [ ]:
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb
%config SqlMagic.displaylimit = 100

In [ ]:
%%sql

create or replace table dam as
select *
from dfp

In [ ]:
%%sql

describe dam

In [ ]:
%%sql

SELECT  indentifier , dataset_title, theme, issued, modified, frequency, count(*) n
FROM dam 
--where year(modified) = 2025
group by all
order by theme asc

In [ ]:
%%sql

SELECT   theme,count(*) n, count(distinct indentifier) d
FROM dam 
--where year(modified) = 2025
group by cube (theme)
order by theme asc 


In [ ]:
%sqlcmd profile --table dam